In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeit import default_timer as timer
import illustris_python as il
import physics

#intitial setup
basePath = "../data/tng-100-3/output"
fields = {"stars": ["Coordinates", "Potential", "Masses", "Velocities"],
        "gas": ["Coordinates", "Potential", "Masses", "Velocities"],
        "dm": ["Coordinates", "Potential", "Velocities"]
        }
indices = [1893, 2164, 3043]

#indices[0] = 1893

#[544, 953, 1285, 1662]
#[1893, 2164, 3043, 3229, 3403, 3605, 3760] 
#np.genfromtxt("../data/tng-100-3/cutdata/central_ids.txt")
DM_PARTICLE_MASS = 0.03235675 #found in header of snapshot
N = len(indices)
stars = [0]*N
gas = [0]*N
dm = [0]*N
particle_lists = [gas, dm, stars]
group_cat = pd.DataFrame({"id": indices})
#Start timer
start = timer()


In [2]:
#Load all particles
for i in range(N):
    stars[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'stars', fields["stars"]))
    gas[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'gas', fields["gas"]))
    dm[i] = il.pandasformat.dict_to_pandas(il.snapshot.loadSubhalo(basePath, 99, indices[i], 'dm', fields["dm"]))
    dm[i]["Masses"] = [DM_PARTICLE_MASS]*len(dm[i]["Potential"]) #Add dm masses

subhaloFields = ["SubhaloMass", 'SubhaloMassType', "SubhaloMassInHalfRadType", "SubhaloHalfmassRadType", "SubhaloPos", "SubhaloVel", "SubhaloSpin"]
subhalos = il.groupcat.loadSubhalos(basePath, 99, fields=subhaloFields)
df_subhalos = il.pandasformat.dict_to_pandas(subhalos)

In [3]:
group_cat = physics.properties.galaxy_radius(group_cat, basePath)
group_cat = physics.properties.center_halo(stars, N, group_cat)
for particle in particle_lists:
    particle = physics.properties.relative_pos_radius(particle, N, group_cat)

for i in range(N):
    max_rad = group_cat["SubhaloGalaxyRad"][i]
    for particle in particle_lists:
        particle[i] = particle[i][particle[i]["r"] < max_rad]

print("Calculating particle masses")
particle_lists, group_cat = physics.properties.total_mass(particle_lists, N, group_cat)

print("Calculating subhalo velocity")
stars, group_cat = physics.properties.subhalo_velocity(stars, N, group_cat)
print("Calculating relative velocities")
stars = physics.properties.relative_velocities(stars, N, group_cat)
print("Calculating half mass radius")
group_cat = physics.properties.half_mass_radius(stars, N, group_cat, "Stellar")

"""
print("Calculating maximum angular momentum")
group_cat = physics.properties.max_ang_momentum(stars, N, group_cat)
rot_vector = np.transpose(np.array([group_cat["RotationAxisX"], group_cat["RotationAxisY"], group_cat["RotationAxisZ"]]))
print("Rotating subhalos")
particle_lists = physics.geometry.rotate_coordinates(particle_lists, N, group_cat, rot_vector)
"""
#End timer
end = timer()

Calculating subhalo velocity
Calculating half mass radius


In [4]:
print("Galaxy radius")
print(group_cat["SubhaloGalaxyRad"])

Galaxy radius
0    83.0704
1    84.4528
2    77.7103
Name: SubhaloGalaxyRad, dtype: object


In [5]:
print("Masses check")
for i in range(2):
    print("Group catalogue: ", df_subhalos["SubhaloMassStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloMassStellar"][i])

Masses check
Group catalogue:  56.558517  Snapshot:  56.55851745605469
Group catalogue:  87.63501  Snapshot:  87.635009765625


In [6]:
print("Velocities check")
for i in range(3):
    print(group_cat["SubhaloVelX"][i], group_cat["SubhaloVelY"][i], group_cat["SubhaloVelZ"][i])
    print(df_subhalos["SubhaloVel"][indices[i]])

Velocities check
-85.89670987401198 123.78385310820924 -67.07539627490819
[ 23.972326  27.339745 -51.660805]
1.765728953975688 -92.23273883649348 -54.0983773748485
[ 36.599586 -97.97224  -68.23334 ]
114.19547997390605 131.49600070528192 85.84660613493374
[54.032154 96.104256 84.92689 ]


In [7]:
print("Halfmass radius check")
for i in range(3): 
    print("Group catalogue: ", df_subhalos["SubhaloHalfmassRadStellar"][indices[i]], " Snapshot: ", group_cat["SubhaloHalfmassRadStellar"][i])

Halfmass radius check
Group catalogue:  22.51544  Snapshot:  22.602636753482404
Group catalogue:  35.24243  Snapshot:  35.24388120202146
Group catalogue:  27.368404  Snapshot:  27.42387717814212


In [8]:
for i in range(N):
    print(stars[i]["r"].max())

83.01346190292674
84.44166469819471
77.49192657189394


In [9]:

#Save all fields for particles in one subhalo for later inspection
#g = 0
#stars[g].to_pickle("../data/tng-100-3/subhalos/subhalo" + str(indices[g]) +"_stars.pkl")

In [10]:
#Save group catalogue
#group_cat.to_pickle("../data/tng-100-3/catalogues/test_cat_01.pkl")

In [11]:

print("Time to process " + str(N) + " Subhalos: ")
print(int(end - start), "Seconds")

Time to process 3 Subhalos: 
7 Seconds
